In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='0b9becfe-d174-4bec-baaf-c7e7fe1b27ab', project_access_token='p-2+TOLfqNSexX1UJg9hxiFMBw==;eixmY9/4DMVcMaB96BxlrQ==:zqIZbFh8THaYsqOl1YPDa83QhqxraD0QhLGIk3MuTQCmb1M/2CcTnyUoW0MARGqgy8c08OdI63KQ4czLR8QDwT5h91+wJKc1JA==')
pc = project.project_context

from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space({'token':'p-2+TOLfqNSexX1UJg9hxiFMBw==;eixmY9/4DMVcMaB96BxlrQ==:zqIZbFh8THaYsqOl1YPDa83QhqxraD0QhLGIk3MuTQCmb1M/2CcTnyUoW0MARGqgy8c08OdI63KQ4czLR8QDwT5h91+wJKc1JA=='})



Set up Project Token and API Key

In [2]:
project.get_name()


'MLOps Monitoring Fraud Detection'

In [3]:
ibmcloud_api_key = 'Apn4to2Oga8LvhspP3ORbatlMqW5EsdUEU-Aorv2jCK8'

Set up

In [4]:
import os
import datetime
import base64
import json
import requests
import time 

from ibm_watson_machine_learning import APIClient
import pandas as pd

import ibm_db, ibm_db_dbi as dbi

User Inputs

WOS Credentials (Watson OpenScale)

In [5]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = IAMAuthenticator(apikey=ibmcloud_api_key)
wos_client = APIClient(authenticator=authenticator)

Load training data

In [6]:
# data = pd.read_csv('./creditcard_2023.csv')

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='kmYxYjVjqDeXgwwK46vUveoIQydIum_5_1CDN3gzJuJ9',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'mlopsmonitoringfrauddetection-donotdelete-pr-d5rj3reapahkjf'
object_key = 'creditcard_2023.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


In [13]:
# Specify a name for the space being created, the saved model and the model deployment
space_name = 'Fraud Detection - Data Science and MLOps'
model_name = 'fraud_detection_best_model'
deployment_name = 'fraud detection model deployment - Data Science'

In [15]:
from ibm_watson_machine_learning import APIClient

location = os.environ['RUNTIME_ENV_REGION'] 
wml_credentials = {
    "apikey": ibmcloud_api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}
wml_client = APIClient(wml_credentials)


l_space_details = []
l_space_details_created_times = []
for space_details in wml_client.spaces.get_details()['resources']:
    if space_details['entity']['name'] == space_name:
        space_id=space_details['metadata']['id']

        try:

            # set this space as default space
            wml_client.set.default_space(space_id)
            print(space_name,"is set as default")
        except:
            space_error=space_name+" not found"
            raise NameError(space_error)
